In [1]:
import pandas as pd
import numpy as np
import pymysql

## 1.读取数据

In [12]:
conn = pymysql.connect(host="localhost",user="root",password="",database="douban",port=3306)
file = pd.read_sql("select pcomment,pvote,pscore from parasite",con=conn)    #从数据库中读取评论数据

In [13]:
file.count()

pcomment    12332
pvote       12332
pscore      12332
dtype: int64

In [14]:
file["pscore"].value_counts()       # 用户对电影的打分

allstar40 rating    3050
allstar10 rating    2455
allstar20 rating    2379
allstar30 rating    2233
allstar50 rating    1980
allstar              235
Name: pscore, dtype: int64

## 2. 数据预处理

In [15]:
file_1 = file[file["pscore"]!="allstar30 rating"]

In [16]:
a = file_1["pscore"]

In [17]:
b = [*map(lambda x: x.split("r")[1].strip(),a)]

In [18]:
def clf_score(str):
    if str in ['50','40']:
        return 1
    else:
        return 0
        

In [19]:
label =  [*map(clf_score,b)]

In [21]:
label.count(1)

5030

In [22]:
texts = file_1["pcomment"].tolist()

In [13]:
texts[0]

'一边是水淹陋室，一边是生日聚会。\n一边是冰冷坚硬的体育馆，一边是豪华柔软的后花园。\n钱就是熨斗，把一切都烫平了。\n他们蹭网、住地下室、为了一份工作不择手段。\n当你觉得他们已经够惨的时候，电影突然反转：他们还不是最惨的，还有更惨的。\n你富，就一定有比你更富的；你穷，也一定有比你更穷的。\n贫富差距就像无数面墙，将人分为了三六九等。\n这面墙，任凭风吹日晒雨淋也岿然屹立，纹丝不动。\n朴夫妇看起来傻乎乎的，可他们还是越来越有钱；\n金基泽一家聪明过人，可还是翻不了身。\n你以为找个好工作，有份好薪水，就摆脱了贫穷的身份，其实还差了十万八千里。\n你能洗掉衣服的汗味，却洗不掉已经融入血液的地下室的气味。\n你爬了十层楼、二十层楼，可能才刚刚到达别人的地下室。\n他们是虫子，所以他们生活艰难；\n他们是虫子，所以什么都杀不死他们。'

In [23]:
import collections,jieba      # 进行分词和去停用词处理

In [24]:
texts_list = [" ".join(jieba.cut(x)).split(" ") for x in texts]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zhuxi\AppData\Local\Temp\jieba.cache
Loading model cost 0.908 seconds.
Prefix dict has been built succesfully.


In [25]:
len(texts_list)

10099

In [113]:
sfile = open(r"D:\NLP\stop_words\stop_words.txt",encoding='utf-8')
stopwords_lst = sfile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

In [27]:
for text in texts_list:
    for word in text:
        if word in stopwords:
            text.remove(word)

In [28]:
word_fre = collections.Counter()
maxlen=0
for i in texts_list:

    if len(i)>maxlen:
        maxlen = len(i)
    for word in i:
        word_fre[word] +=1
    

In [29]:
maxlen    # 短评最大长度

256

In [30]:
len(word_fre)   # 语料库词数

35148

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x_train,x_test,ytrain,ytest =train_test_split(texts_list,label,test_size=0.3,random_state=42)

## 3. 建模分析
## 神经网络

In [33]:
from keras import preprocessing

C:\Users\zhuxi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [34]:
maxlen = 250

In [35]:
from keras.preprocessing.text import Tokenizer 

In [36]:
token = Tokenizer(num_words=35000)

In [37]:
token.fit_on_texts(x_train)

In [38]:
train_num = token.texts_to_sequences(x_train)
test_num = token.texts_to_sequences(x_test)

In [51]:
train_num[0]

[128,
 989,
 647,
 81,
 1328,
 3,
 105,
 933,
 479,
 933,
 371,
 33,
 2,
 933,
 46,
 843,
 5670,
 132,
 1897,
 67,
 4,
 3]

In [39]:
xtrain = preprocessing.sequence.pad_sequences(train_num,maxlen=maxlen)
xtest = preprocessing.sequence.pad_sequences(test_num,maxlen=maxlen)

In [40]:
from keras.layers import Flatten,Dense,Embedding, LSTM
from keras.models import Sequential

### 包含embedding层和Dense层的模型

In [55]:
model = Sequential()
model.add(Embedding(30000,128,input_length=250))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 250, 128)          3840000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1024032   
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 4,864,065
Trainable params: 4,864,065
Non-trainable params: 0
_________________________________________________________________


In [56]:
history = model.fit(xtrain,ytrain,epochs=20,batch_size=128,validation_data=(xtest,ytest))

Train on 7069 samples, validate on 3030 samples
Epoch 1/20
7069/7069 [==============================] - 6s 898us/step - loss: 0.6694 - acc: 0.6325 - val_loss: 0.5204 - val_acc: 0.8036
Epoch 2/20
7069/7069 [==============================] - 5s 722us/step - loss: 0.3493 - acc: 0.8625 - val_loss: 0.3665 - val_acc: 0.8422
Epoch 3/20
7069/7069 [==============================] - 5s 719us/step - loss: 0.1740 - acc: 0.9352 - val_loss: 0.3623 - val_acc: 0.8528
Epoch 4/20
7069/7069 [==============================] - 5s 718us/step - loss: 0.0976 - acc: 0.9670 - val_loss: 0.3660 - val_acc: 0.8564
Epoch 5/20
7069/7069 [==============================] - 5s 718us/step - loss: 0.0547 - acc: 0.9819 - val_loss: 0.3973 - val_acc: 0.8617
Epoch 6/20
7069/7069 [==============================] - 5s 742us/step - loss: 0.0300 - acc: 0.9902 - val_loss: 0.4533 - val_acc: 0.8578
Epoch 7/20
7069/7069 [==============================] - 6s 779us/step - loss: 0.0159 - acc: 0.9966 - val_loss: 0.5265 - val_acc: 0.8521


### 包含emdedding层和LSTM层

In [45]:
from keras import regularizers

In [53]:
model1 = Sequential()
model1.add(Embedding(30000,256,input_length=250))
model1.add(LSTM(32,dropout=0.3,recurrent_dropout=0.2))
# model1.add(LSTM(32,dropout=0.3,recurrent_dropout=0.2))
model1.add(Dense(1,activation='sigmoid',kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.001)))
model1.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 250, 256)          7680000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                36992     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 7,717,025
Trainable params: 7,717,025
Non-trainable params: 0
_________________________________________________________________


In [58]:
history = model1.fit(xtrain,ytrain,epochs=10,batch_size=128,validation_data=(xtest,ytest))

Train on 7069 samples, validate on 3030 samples
Epoch 1/10
7069/7069 [==============================] - 32s 5ms/step - loss: 0.1101 - acc: 0.9919 - val_loss: 0.6077 - val_acc: 0.8564
Epoch 2/10
7069/7069 [==============================] - 37s 5ms/step - loss: 0.1064 - acc: 0.9939 - val_loss: 0.6346 - val_acc: 0.8502
Epoch 3/10
7069/7069 [==============================] - 39s 6ms/step - loss: 0.1019 - acc: 0.9942 - val_loss: 0.6645 - val_acc: 0.8488
Epoch 4/10
7069/7069 [==============================] - 37s 5ms/step - loss: 0.0990 - acc: 0.9952 - val_loss: 0.6396 - val_acc: 0.8479
Epoch 5/10
7069/7069 [==============================] - 38s 5ms/step - loss: 0.0976 - acc: 0.9949 - val_loss: 0.6483 - val_acc: 0.8459
Epoch 6/10
7069/7069 [==============================] - 37s 5ms/step - loss: 0.0941 - acc: 0.9962 - val_loss: 0.6315 - val_acc: 0.8515
Epoch 7/10
7069/7069 [==============================] - 36s 5ms/step - loss: 0.0918 - acc: 0.9967 - val_loss: 0.6415 - val_acc: 0.8485
Epoch 8

In [57]:
model.predict(xtest)

array([[2.1129847e-05],
       [9.9969923e-01],
       [4.0531158e-06],
       ...,
       [1.4315336e-02],
       [8.9548033e-01],
       [7.3639864e-01]], dtype=float32)

In [49]:
from sklearn.metrics import roc_auc_score

In [57]:
roc_auc_score(ytest,model.predict(xtest))    # 模型的roc值

0.9114033784334342

# word2vec

In [61]:
import gensim

model = gensim.models.Word2Vec(texts_list,size=200,sg=1,min_count=1)

a=model.wv.vectors

In [60]:
def makeFeatureVec(text,model,num_features):
    featureVec = np.zeros((num_features,),dtype='float32')
    nwords=0.
    index2word_set = set(model.wv.index2word)
    for word in text:
        if word in index2word_set:
            nwords +=1
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getTextsVec(texts,model,num_features):
    counter = 0
    textsVec = np.zeros((len(texts),num_features),dtype='float32')
    
    for text in texts:
        textsVec[counter] = makeFeatureVec(text,model,num_features)
        counter +=1
    return textsVec

In [62]:
xtrainVec = getTextsVec(x_train,model,200)
xtestVec = getTextsVec(x_test,model,200)

C:\Users\zhuxi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\zhuxi\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [75]:
xtrainVec = pd.DataFrame(xtrainVec)
tra_vec = xtrainVec.fillna(0)

In [76]:
tra_vec.shape

(7069, 200)

In [77]:
xtestVec = pd.DataFrame(xtestVec)
tes_vec = xtestVec.fillna(0)

In [78]:
tes_vec.shape

(3030, 200)

In [63]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as rf

In [79]:
clf = GaussianNB()
clf.fit(tra_vec,ytrain)
clf.score(tes_vec,ytest)

0.63993399339934

In [82]:
rfc = rf(n_estimators=300)
cross_val_score(rfc,tra_vec,ytrain,cv=5)

array([0.78429986, 0.7758133 , 0.76874116, 0.77298444, 0.77140835])

# Tf-idf

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [128]:
text_lists = [" ".join(jieba.cut(x)) for x in texts]

In [138]:
xtrain,xval,y_train,y_val =train_test_split(text_lists,label,test_size=0.3,random_state=420)

In [130]:
xtrain[0]

'部分 黑色幽默 。 \n 认为 三个 小时 完全 Ok ， 满满 料 。 \n “ 前进 了 一步 ， 后退 了 两步 。 ” 我 喜欢 他们 的 互相理解 。 \n 被 世界 抛弃 的 阿姨 生不逢时 。 \n 好美   ... \n'

In [118]:
tf = TfidfVectorizer(stop_words=stopwords,max_features=3000, ngram_range=(1,2),lowercase=False)

In [133]:
tf.fit(xtrain)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=3000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [139]:
xtra=tf.transform(xtrain).toarray()
xval = tf.transform(xval).toarray()

In [140]:
clf = GaussianNB()
clf.fit(xtra,y_train)
clf.score(xval,y_val)

0.7940594059405941

### 对于对电影短评的情感分析，对于短评的前期处理，首先采用jieba分词，然后分别采用了word2vec和tf-idf得到词向量，之后利用sklearn中的朴素贝叶斯和随机森林算法建模，分类结果均较差。对于LSTM和神经网络建模，由于模型中添加了embedding层，因此在预处理时未得到词向量，LSTM的分类结果略优于神经网络，但计算耗时却是普通神经网络的7倍左右，两类模型的auc值均在0.86左右，模型均有待提高，目前评论数据中，正类和负类样品数均为5000左右，样本数据偏少，这是模型过拟合的原因之一。对于朴素贝叶斯算法的结果，需要进一步分析研究。